1. 특정 업종내 종목별 변동성 시각화
2. 특정 업종내 종목별 수익률 시각화

In [1]:
# 데이터 분석을 위한 패키지
import numpy as np
import pandas as pd

In [2]:
# 시각화 패키지
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import FinanceDataReader as fdr

In [4]:
import koreanize_matplotlib

In [5]:
import datetime
from dateutil.relativedelta import relativedelta

In [6]:
import plotly.express as px

In [7]:
from data_loader import load_sector_data, get_market_color

In [8]:
# 모든 컬럼,행을 출력하도록 설정
pd.set_option('display.max_columns', None) # None으로 설정하면 모든 컬럼 출력
pd.set_option('display.max_rows', None) #None으로 설정하면 모든 행 출력

In [ ]:
# 기존 그래프
#   plt.figure(figsize=(10, 20))
#   sns.barplot(y='Name', x='Volatility', data=sector_data)
#   # 평균 변동성을 기준으로 수평선 추가
#   plt.axvline(avg_volatility, color='red', linestyle='--', label=f'평균 변동성: {avg_volatility:.2f}')
#   # 평균 변동성 수치 표시
#   plt.text(avg_volatility + 0.1, 10, f'평균 변동성 : {avg_volatility:.2f}%', color='red', va='center', fontsize=10)
#   plt.title(f'{sector} 업종 내 종목 변동성')
#   plt.xlabel('변동성')
#   plt.ylabel('종목명')
#   plt.show()

업종내 종목들의 변동성을 시각화 후 업종별로 정렬
hist 혹은 bar 형태?

In [9]:
# 특정 업종내 종목별 변동성 시각화
def get_volatility_by_stock(sector, market=None, month_ago=3) :
  '''
  특정 업종 내 종목별 변동성을 시각화하고, 해당 업종의 평균 변동성을 출력합니다.

  Parameters:
  sector (str): 분석할 업종의 이름.

  Returns:
  None: 이 함수는 그래프를 시각화하고, 평균 변동성 및 관련 데이터를 출력합니다.

  Example:
  >>> get_volatility_by_stock('IT서비스', 'KOSPI')
  '''
  # 시장 데이터 로드
  df = load_sector_data(market, month_ago=month_ago)
  
  sector_data = df[df['Sector'] == sector].sort_values(by='Volatility', ascending=True)
  
  # 해당 업종의 평균 변동성 계산
  avg_volatility = df[df['Sector'] == sector]['Volatility'].mean()
  
  color = get_market_color(market)
  
  # Figure의 세로 사이즈 조정 (업종수에 따라)
  figure_height = 400 + (len(sector_data) * 30) 
  
  # Plotly를 사용한 대화형 바 차트 생성
  fig = px.bar(sector_data, 
                    x='Volatility', 
                    y='Name',
                    color_discrete_sequence=[color], 
                    title=f'{sector} 업종 내 종목 {month_ago}개월 평균 변동성',
                    labels={'Volatility': '변동성 (%)', 'Name': '종목명'},
                    text='Volatility',
                    height=figure_height)

  fig.add_vline(x=avg_volatility, line_dash="dash", line_color="red", 
                  annotation_text=f'평균 변동성: {avg_volatility:.2f}%', 
                  annotation_position="top right")
  
  print(f'{sector} 업종의 변동성 평균: {avg_volatility:.2f}%')
  # display(sector_data)

In [12]:
get_volatility_by_stock('IT서비스', market='KOSPI')

3개월 간의 data를 불러옵니다.


IT서비스 업종의 변동성 평균: 3.17%


In [13]:
get_volatility_by_stock('채권', market='ETF')

3개월 간의 data를 불러옵니다.


채권 업종의 변동성 평균: 0.30%


In [14]:
get_volatility_by_stock('반도체와반도체장비', market='KOSPI')

3개월 간의 data를 불러옵니다.


반도체와반도체장비 업종의 변동성 평균: 3.32%


In [35]:
# 특정 업종내 종목별 수익률 시각화

def get_return_by_stock(sector, market, month_ago=3) :
  df = load_sector_data(market,month_ago=month_ago)
  sector_data = df[df['Sector'] == sector].sort_values(by='TotalReturn', ascending=True)
  
  color = get_market_color(market)
  
  avg_market_total_return = df['TotalReturn'].mean()
  avg_sector_total_return = sector_data['TotalReturn'].mean()
  
  # Figure의 세로 사이즈 조정 (업종수에 따라)
  figure_height = 400 + (len(sector_data) * 30)
  
  fig = px.bar(
        x=sector_data['TotalReturn'],
        y=sector_data['Name'],
        color_discrete_sequence=[color],
        title=f'{market} {sector} 업종별 {month_ago}개월 평균 수익률',
        labels={'x':'평균 수익률 (%)','y':'종목명'}
    )
  
  # y축의 최대값과 최소값
  y_max = len(sector_data) - 0.5
  y_min = -0.5
  
  # 평균 수직선 추가
  fig.add_shape(type='line', xref='x', yref='y',
                x0=avg_market_total_return, y0=-0.5, x1=avg_market_total_return, y1=len(sector_data)-0.5,
                line=dict(color='blue', dash='dash'),
                name=f'{market} 평균 수익률(%)')
    
  # 평균 라벨 추가
  fig.add_annotation(
        x=avg_market_total_return,
        y=y_max - 1.5,
        text=f'{market} 평균 수익률(%) : {avg_market_total_return:.3f} %',
        showarrow=True,
        arrowhead=2,
        ax=0,
        ay=-40,
        font=dict(color='blue')
    )
  # 평균 수직선 추가
  fig.add_shape(type='line', xref='x', yref='y',
                x0=avg_sector_total_return, y0=-0.5, x1=avg_sector_total_return, y1=len(sector_data)-0.5,
                line=dict(color='orange', dash='dash'),
                name=f'{market} 평균 수익률(%)')
    
  # 평균 라벨 추가
  fig.add_annotation(
        x=avg_sector_total_return,
        y= -len(sector_data),
        text=f'{market} 평균 수익률(%) : {avg_sector_total_return:.3f} %',
        showarrow=True,
        arrowhead=2,
        ax=0,
        ay=40,
        font=dict(color='blue')
    )
  
  # 레이아웃 조정
  fig.update_layout(
        showlegend=True,
        height=figure_height
  )

  fig.show()
  
   
  # plt.figure(figsize=(10, 6))
  # sns.barplot(y='Name', x='TotalReturn', data=sector_data)
  
  # # 해당 업종의 평균 수익률 계산
  
  
  # # 평균 수익률을 기준으로 수평선 추가
  # plt.axvline(avg_total_return, color='red', linestyle='--', label=f'평균 수익률: {avg_total_return:.2f}')
  
  # # 평균 수익률 수치 표시
  # plt.text(avg_total_return + 0.1, 10, f'평균 수익률 : {avg_total_return:.2f}%', color='red', va='center', fontsize=10)
  
  # plt.title(f'{sector} 업종 내 종목 {month_ago}개월 수익률 평균')
  # plt.xlabel('수익률')
  # plt.ylabel('종목명')
  # plt.show()
  
  
  # print(f'{sector} 업종의 수익률 평균: {avg_total_return:.2f}%')
  # display(sector_data)
  

In [36]:
get_return_by_stock('반도체와반도체장비', 'KOSPI', month_ago=3)

3개월 간의 data를 불러옵니다.


In [17]:
get_return_by_stock('반도체와반도체장비', 'KOSPI', month_ago=3)

3개월 간의 data를 불러옵니다.
